In [ ]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import pymysql
pymysql.install_as_MySQLdb()

In [ ]:
#Import Datasets
#Draft Data
draft_file = "./Datasets/draft78.csv"
draft_df = pd.read_csv(draft_file)
#Salery Data
salary_file = "./Datasets/Player - Salaries per Year (1990 - 2017).xlsx"
salary_df = pd.read_excel(salary_file)
#Player Data
player_file = "./Datasets/player_data.csv"
player_df = pd.read_csv(player_file)

In [ ]:
#Cleaning player_df and draft_df
player_df = player_df.drop(columns=['year_start', 'year_end'])
draft_df = draft_df.drop(columns=['Yrs'])

In [ ]:
#Merging Salary and Draft
salary_df_2 = pd.merge(salary_df,  draft_df, how='inner', left_on='Player Name', 
                       right_on='Player', copy=False)
salary_df_2 = salary_df_2.drop(columns=['Player'])

In [ ]:
#Merging New Salary Dataframe with Player Info and dropping unnecessary
final_df = pd.merge(salary_df_2, player_df, left_on='Player Name', right_on='name')
final_df = final_df.drop(columns=['Register Value','Team', 'name'])

In [ ]:
#Renaming Columns to dump into SQL DB
final_df = final_df.rename(columns={
    'Player Name':'player_name', 'Salary in $': 'salary', 'Season Start': 'season_start',
    'Season End':'season_end', 'Full Team Name': 'team_name', 'Pick':'draft_rank',
    'Draft':'draft_name'
})
final_df.head()

In [ ]:
#Setting up connection to SQL
connection_string = "root:S2hockey@localhost/nba_db"
engine = create_engine(f'mysql://{connection_string}')
engine.table_names()

In [ ]:
#Load Data into SQL
final_df.to_sql(name='nba_info', con=engine, if_exists='append', index=False)

In [ ]:
#Create new dataframe for analysis
final_a_df = final_df.groupby('player_name').max()
final_a_df.head()

In [ ]:
#Converting height to inches
heights = final_a_df['height'].str.split('-', n = 1, expand = True)
heights = heights[[0,1]].astype('int64')
heights['height_inches'] = 12*heights[0] + heights[1]
heights = heights[['height_inches']]

In [ ]:
#Pulling Just the Salaries
just_salaries = final_a_df[['salary']]
#Merging with height table
heights1 = pd.merge(heights, just_salaries, on='player_name')
#Creating new table with average salaries grouped by height in inches

In [ ]:
#Creating Linear Regression for Height in Inches vs. Max Salary for all players

#Importing Stats and Matplotlib
from scipy.stats import linregress
import matplotlib.pyplot as plt
#Making Line of Best Fit
(slope, intercept, _, _, _) = linregress(heights1['height_inches'], heights1['salary'])
fit = slope * heights1['height_inches'] + intercept
#Making Plot
#Making plot
fig, ax = plt.subplots()
fig.suptitle("Player Height in Inches vs. Max Salary Recieved", fontsize=12)
ax.set_xlabel("Player Height (inches)")
ax.set_ylabel("Max Salary ($)")
ax.plot(heights1['height_inches'], heights1['salary'], color = 'red', linewidth=0, marker='o')
ax.plot(heights1['height_inches'], fit, 'b--')
plt.grid()
plt.show()
corr_coef = np.corrcoef(heights1['height_inches'], heights1['salary'])[0, 1]
print('The slope of the line of best fit is ' + str(slope))
print(f'The correlation coefficient is {corr_coef}')

In [ ]:
#Creating Linear Regression for Height in Inches vs. Max Salary grouped by Height
#Making Line of Best Fit
(slope2, intercept2, _, _, _) = linregress(heights2.index.values, heights2['salary'])
fit2 = slope2 * heights2.index.values + intercept2
#Making Plot
#Making plot
fig2, ax2 = plt.subplots()
fig2.suptitle("Player Height in Inches vs. Average Max Salary Recieved", fontsize=12)
ax2.set_xlabel("Player Height (inches)")
ax2.set_ylabel("Average Max Salary ($)")
ax2.plot(heights2.index.values, heights2['salary'], color = 'red', linewidth=0, marker='o')
ax2.plot(heights2.index.values, fit2, 'b--')
plt.grid()
plt.show()
corr_coef2 = np.corrcoef(heights2.index.values, heights2['salary'])[0, 1]
print('The slope of the line of best fit is ' + str(slope2))
print(f'The correlation coefficient is {corr_coef2}')

In [ ]:
#Making a draft table
draft_rank = final_a_df[['draft_rank', 'salary']]
draft_rank = draft_rank.groupby('draft_rank').mean()
draft_rank.head()

In [ ]:
#Making linear regression for draft rank vs. average salary
#Making Line of Best Fit
(slope3, intercept3, _, _, _) = linregress(draft_rank.index.values, draft_rank['salary'])
fit3 = slope3 * draft_rank.index.values + intercept3
#Making Plot
#Making plot
fig3, ax3 = plt.subplots()
fig3.suptitle("Draft Rank vs. Average Max Salary Recieved", fontsize=12)
ax3.set_xlabel("Draft Rank")
ax3.set_ylabel("Average Max Salary ($)")
ax3.plot(draft_rank.index.values, draft_rank['salary'], color = 'red', linewidth=0, marker='o')
ax3.plot(draft_rank.index.values, fit3, 'b--')
plt.grid()
plt.show()
corr_coef3 = np.corrcoef(draft_rank.index.values, draft_rank['salary'])[0, 1]
print('The slope of the line of best fit is ' + str(slope3))
print(f'The correlation coefficient is {corr_coef3}')

In [ ]:
#Salary vs Position Analysis
group_by_pos = final_df.groupby('position').mean()

#Plot Average Rating By State
plt.figure(figsize = (20,10))
plt.bar(np.arange(0,len(group_by_pos),1),height = group_by_pos['salary'], color = 'lightblue', align = 'center', width = 0.8,)
plt.grid()
plt.title("Position vs. Average Salary", fontsize = 20)
plt.xlabel("Position", fontsize = 15)
plt.ylabel("Average Salary (in Millions)", fontsize = 15)
plt.xticks(np.arange(0,len(group_by_pos),1),('C','C-F','F','F-C','F-G','G','G-F'))

In [ ]:
#Average Salary by Position over time Analysis

#Create years series
years = np.arange(1991,2019,1)
#Create new data frame
final_df1 = final_df.groupby(['position','season_end']).mean()
#Create series for each position
center = final_df1.loc['C',['salary']]
center_forward = final_df1.loc['C-F',['salary']]
forward = final_df1.loc['F',['salary']]
forward_center = final_df1.loc['F-C',['salary']]
forward_guard = final_df1.loc['F-G',['salary']]
guard = final_df1.loc['G',['salary']]
guard_forward = final_df1.loc['G-F',['salary']]

In [ ]:
#Plot Line Graph

plt.figure(figsize = (20,10))

plt.plot(years,center['salary'],color='green',label='Center')
plt.plot(years,center_forward['salary'],color='blue',label='Center_Forward')
plt.plot(years,forward['salary'],color='red',label='Forward')
plt.plot(years,forward_center['salary'],color='orange',label='Forward_Center')
plt.plot(years,forward_guard['salary'],color='magenta',label='Forward_Guard')
plt.plot(years,guard['salary'],color='black',label='Guard')
plt.plot(years,guard_forward['salary'],color='cyan',label='Guard_Forward')

plt.grid()
plt.legend(loc="best")
plt.title("Average Salary by Position over Time", fontsize = 20)
plt.xlabel("Years", fontsize = 15)
plt.ylabel("Salary (in Millions)", fontsize = 15)